In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Nilesh\\Documents\\GitHub\\End to End ClearSight AI\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Nilesh\\Documents\\GitHub\\End to End ClearSight AI'

In [5]:
os.chdir("src")

In [6]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 666A-2AE9

 Directory of c:\Users\Nilesh\Documents\GitHub\End to End ClearSight AI\src

17-04-2024  14:01    <DIR>          .
17-04-2024  14:50    <DIR>          ..
17-04-2024  12:47    <DIR>          clearSightAI
17-04-2024  12:40    <DIR>          clearSightAI.egg-info
17-04-2024  14:01    <DIR>          logs
               0 File(s)              0 bytes
               5 Dir(s)  286,943,866,880 bytes free


In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [8]:
from clearSightAI.constants import *
from clearSightAI.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [11]:
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Multiply
from tensorflow.keras.models import Model

In [12]:

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    def update_base_model(self):
        if self.model is None:
            raise ValueError("No custom model loaded. Please load a model first.")

        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        # Add attention mechanism
        attention_output = PrepareBaseModel.spatial_attention(model.get_layer('block3_conv3').output)


        flatten_in = tf.keras.layers.Flatten()(attention_output)

        dense1 = Dense(128, activation='relu')(flatten_in)  # Reduced number of units

        output = Dense(1, activation='sigmoid')(dense1)

        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(output)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    @staticmethod
    def spatial_attention(input_feature):
        # Compute attention map
        attention = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(input_feature)
        # Multiply attention map with input feature
        weighted_feature = Multiply()([input_feature, attention])
        return weighted_feature

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [13]:
os.chdir("../")
%ls

 Volume in drive C is Windows
 Volume Serial Number is 666A-2AE9

 Directory of c:\Users\Nilesh\Documents\GitHub\End to End ClearSight AI

17-04-2024  14:50    <DIR>          .
17-04-2024  12:13    <DIR>          ..
17-04-2024  12:17    <DIR>          .github
17-04-2024  15:37    <DIR>          artifacts
17-04-2024  12:17    <DIR>          config
17-04-2024  12:17                 0 dvc.yaml
17-04-2024  12:47    <DIR>          logs
17-04-2024  14:49               459 main.py
17-04-2024  15:11               168 params.yaml
17-04-2024  12:39                 0 README.md
17-04-2024  12:40               194 requirements.txt
17-04-2024  15:12    <DIR>          research
17-04-2024  12:37             1,066 setup.py
17-04-2024  14:01    <DIR>          src
17-04-2024  12:17             1,327 template.py
17-04-2024  12:17    <DIR>          templates
               7 File(s)          3,214 bytes
               9 Dir(s)  286,943,731,712 bytes free


In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-04-17 15:43:13,646: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-17 15:43:13,650: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-17 15:43:13,657: INFO: common: created directory at: artifacts]
[2024-04-17 15:43:13,660: INFO: common: created directory at: artifacts/prepare_base_model]


[2024-04-17 15:43:14,004: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 224, 224,  │      1,792 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 224, 224,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 112, 112,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 112, 112,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 112, 112,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 56, 56,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 56, 56,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 56, 56,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 56, 56,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 56, 56, 1) │        257 │ block3_conv3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 56, 56,    │          0 │ block3_conv3[0][… │
│                     │ 256)              │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 802816)    │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │ 102,760,5… │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 2)         │          4 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 104,496,454 (398.62 MB)

 Trainable params: 102,760,966 (392.00 MB)

 Non-trainable params: 1,735,488 (6.62 MB)

[2024-04-17 15:43:14,794: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
